# 크롤링 연습

[DB다이렉트 홈페이지](https://www.directdb.co.kr/mainView.do)

[노션에 끄적인 거](https://bony-mousepad-5b9.notion.site/DB-1ce8496cb546808fa193f84c43e5726c?pvs=4)

In [2]:
import requests

# url = "https://www.directdb.co.kr/mainView.do"
url = "https://kpub.knia.or.kr/carInsuranceDisc/specialTerms/specialTerms.do"
res = requests.get(url, verify = False)
db_html = res.text

C:\Users\Playdata\AppData\Local\miniconda3\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'kpub.knia.or.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [3]:
import lxml
from bs4 import BeautifulSoup

soup = BeautifulSoup(db_html, 'lxml')
# print(soup)

In [4]:
table_div = soup.find_all('table')[1]
print(table_div.prettify())

<table class="fixTable" data-tddepth="0">
 <caption>
  자동차보험 할인형 특별약관 조회 결과
 </caption>
 <colgroup>
  <col style="width: 140px;"/>
  <col style="width: 100px;"/>
  <col style="width: 230px;"/>
  <col style="width: auto;"/>
  <col style="width: 140px;"/>
  <col style="width: 180px;"/>
  <col style="width: 80px;"/>
 </colgroup>
 <thead>
  <tr>
   <th rowspan="2" scope="col">
    구분
   </th>
   <th rowspan="2" scope="col">
    회사명
   </th>
   <th rowspan="2" scope="col">
    특약명
   </th>
   <th rowspan="2" scope="col">
    가입조건
   </th>
   <th rowspan="2" scope="col">
    판매채널
   </th>
   <th colspan="2" scope="colgroup">
    할인율
   </th>
  </tr>
  <tr>
   <th scope="col">
    적용담보
    <br/>
    (자동차상해 및
    <br/>
    담보별 확장특약 포함기준)
   </th>
   <th scope="col">
    할인율(%)
    <span class="common_sort_btns">
     <button class="sort_asc" type="button">
      <em>
       asc
      </em>
     </button>
     <button class="sort_desc" type="button">
      <em>
       desc
      </em>
     </bu

In [5]:
import pandas as pd

thead_rows = table_div.find("thead").find_all("tr")
headers = []

for th in thead_rows[0].find_all("th"):
    text = th.get_text(strip=True)
    colspan = th.get("colspan")
    if colspan:
        headers.extend([text] * int(colspan))
    else:
        headers.append(text)

# 두 번째 줄 헤더 반영
sub_headers = [th.get_text(strip=True) for th in thead_rows[1].find_all("th")]

# 헤더 병합
print(headers)
print(sub_headers)
headers[5] = sub_headers[0]
headers[-1] = sub_headers[1]
headers


# 3. 데이터 추출
data_rows = []
for row in table_div.find("tbody").find_all("tr"):
    cols = [td.get_text(strip=True) for td in row.find_all(["td", "th"])]
    data_rows.append(cols)

# 4. DataFrame으로 만들기
df = pd.DataFrame(data_rows, columns=headers)

# 결과 확인
df



['구분', '회사명', '특약명', '가입조건', '판매채널', '할인율', '할인율']
['적용담보(자동차상해 및담보별 확장특약 포함기준)', '할인율(%)ascdesc']


,구분,회사명,특약명,가입조건,판매채널,적용담보(자동차상해 및담보별 확장특약 포함기준),할인율(%)ascdesc
0,자녀할인,메리츠화재,자녀(만7세이하) 할인 특별약관,▶ 다음의 조건 모두 해당하는 경우에 한하여 가입 가능① 자녀(태아) 연령조건 : ...,대면/TM/CM/PM,대인1/대인2/대물/자손/무보험/자차,7
1,자녀할인,한화손보,자녀할인 특별약관,▶ 다음의 조건 모두 해당하는 경우에 한하여 가입 가능① 자녀(태아) 연령조건 : ...,"대면, TM","대인1, 대인2, 대물, 자손, 무보험차상해, 자차",24.4
2,자녀할인,한화손보,자녀할인 특별약관,▶ 다음의 조건 모두 해당하는 경우에 한하여 가입 가능① 자녀(태아) 연령조건 : ...,"대면, TM","대인1, 대인2, 대물, 자손, 무보험차상해, 자차",20.1
3,자녀할인,한화손보,자녀할인 특별약관,▶ 다음의 조건 모두 해당하는 경우에 한하여 가입 가능① 자녀(태아) 연령조건 : ...,"대면, TM","대인1, 대인2, 대물, 자손, 무보험차상해, 자차",20.6
4,자녀할인,한화손보,자녀할인 특별약관,▶ 다음의 조건 모두 해당하는 경우에 한하여 가입 가능① 자녀(태아) 연령조건 : ...,"대면, TM","대인1, 대인2, 대물, 자손, 무보험차상해, 자차",16.1
...,...,...,...,...,...,...,...
410,서민(나눔)우대할인,KB손보,희망나누기 특별약관,(상동),CM/PM,대인1/대인2/대물/자손/무보험/자차,1
411,서민(나눔)우대할인,DB손보,프로미하트(서민우대 보험료 할인) 특별약관,(상동),대면,대인1/대인2/대물/\n자손/무보험/자차,8
412,서민(나눔)우대할인,DB손보,프로미하트나눔(서민우대 보험료 할인) 특별약관,(상동),TM/CM/플랫폼,대인2/대물/\n자손/무보험/자차,5.4
413,서민(나눔)우대할인,DB손보,프로미하트나눔(서민우대 보험료 할인) 특별약관,(상동),TM/CM/플랫폼,대인1,3.4


In [7]:
df.to_csv("자동차보험_할인형_특별약관.csv", index=False, encoding='utf-8-sig')

In [9]:
%pip install streamlit

   ---------------------------------------- 0.0/9.8 MB ? eta -:--:--
   -- ------------------------------------- 0.5/9.8 MB 3.4 MB/s eta 0:00:03
   ---- ----------------------------------- 1.0/9.8 MB 3.0 MB/s eta 0:00:03
   ------ --------------------------------- 1.6/9.8 MB 3.0 MB/s eta 0:00:03
   -------- ------------------------------- 2.1/9.8 MB 2.7 MB/s eta 0:00:03
   --------- ------------------------------ 2.4/9.8 MB 2.6 MB/s eta 0:00:03
   ----------- ---------------------------- 2.9/9.8 MB 2.2 MB/s eta 0:00:04
   ------------ --------------------------- 3.1/9.8 MB 2.1 MB/s eta 0:00:04
   ---------------- ----------------------- 3.9/9.8 MB 2.3 MB/s eta 0:00:03
   ------------------ --------------------- 4.5/9.8 MB 2.4 MB/s eta 0:00:03
   --------------------- ------------------ 5.2/9.8 MB 2.5 MB/s eta 0:00:02
   ----------------------- ---------------- 5.8/9.8 MB 2.5 MB/s eta 0:00:02
   ------------------------- -------------- 6.3/9.8 MB 2.5 MB/s eta 0:00:02
   ----------------

In [10]:
import streamlit as st
import pandas as pd

# CSV 데이터 로드
@st.cache_data
def load_data():
    return pd.read_csv("자동차보험_할인형_특별약관.csv")

df = load_data()
df.columns = ['구분', '회사명', '특약명', '가입조건', '판매채널', '적용담보', '할인율']

st.title("🚗 자동차 보험 할인 특약 조회 시스템")

# 카테고리(구분) 선택
category = st.selectbox("할인 특약 구분을 선택하세요:", df['구분'].unique())

# 해당 구분 필터링
filtered_df = df[df['구분'] == category]

# 회사별로 출력
for company in filtered_df['회사명'].unique():
    st.subheader(f"🏢 {company}")
    st.dataframe(
        filtered_df[filtered_df['회사명'] == company][['특약명', '가입조건', '판매채널', '적용담보', '할인율']],
        use_container_width=True
    )


2025-04-08 20:58:50.268 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-04-08 20:58:50.273 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-04-08 20:58:50.274 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-08 20:58:50.930 
  command:

    streamlit run C:\Users\Playdata\AppData\Local\miniconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-04-08 20:58:50.930 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-08 20:58:50.942 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-08 20:58:50.998 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-08 20:58:51.002 Thread 'M